**Install Required Packages**

In [0]:
%cd ~

!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-tk

!pip3 install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip3 install -q pycocotools

%cd ~/models/research
!protoc object_detection/protos/*.proto --python_out=.

!python3 setup.py build
!python3 setup.py install

%cd slim
!pip3 install -e .

%cd ..
!python3 object_detection/builders/model_builder_test.py

**Mount to Google Drive**

In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [22]:
%cd ~/models/research/object_detection
!mkdir training
%cd training
from google.colab import files
uploaded = files.upload()

/root/models/research/object_detection
mkdir: cannot create directory ‘training’: File exists
/root/models/research/object_detection/training


Saving ssd_mobilenet_v2.config to ssd_mobilenet_v2.config


**Re-Train Model**

In [0]:
%cd ~/models/research/object_detection
!rm -r ~/models/research/object_detection/training/training/

!python3 model_main.py \
    --pipeline_config_path=training/ssd_mobilenet_v2.config \
    --model_dir=training/training \
    --alsologtostderr \

/root/models/research/object_detection
W0826 02:50:59.817236 140603710080896 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0826 02:50:59.841549 140603710080896 deprecation_wrapper.py:119] From /root/models/research/slim/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0826 02:50:59.850792 140603710080896 deprecation_wrapper.py:119] From /root/models/research/slim/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0826 02:50:59.860885 140603710080896 deprecation_wrapper.py:119] From model_main.py:109: The name tf.app.run is deprec

**Save Model and Convert to .tflite**

In [0]:
%cd ~/models/research/object_detection/training
!mkdir saved
%cd saved
!mkdir original
!mkdir converted
%cd ..
%cd ..
!python3 export_tflite_ssd_graph.py \
    --pipeline_config_path=training/ssd_mobilenet_v2.config \
    --trained_checkpoint_prefix=training/training/model.ckpt-9435 \
    --output_directory=training/saved/original \
    --add_postprocessing_op=true
!toco \
    --graph_def_file=training/saved/original/tflite_graph.pb \
    --output_file=training/saved/converted/converted.tflite \
    --input_shapes=1,600,800,3 \
    --input_arrays=normalized_input_image_tensor \
    --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  \
    --inference_type=FLOAT \
    --allow_custom_ops